In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
from matplotlib import animation

In [ ]:
class particle():
    def __init__(self, position, direction):
        self.p = np.array(position)
        self.d = direction
        self.m = 1
        self.dt = 0.25 #s
        self.initrho = 1
        self.rho = 0
        self.pre = 0
        self.vmax = 10
        self.a = np.array([0, 0, 0])
        self.g = np.array([0, 0, 9.81])
    
    def get_magnitude(self):
        return np.linalg.norm(self.p)
    
    def get_pos(self):
        return self.p
    
    def set_pos(self, new_pos):
        self.p = new_pos

    def get_dir(self):
        return self.d
    
    def get_pre(self):
        return self.pre
    
    def get_acc(self):
        return self.a
    
    def get_m(self):
        return self.m
    
print("Particle class built!")

In [ ]:
class particle_array():
    def __init__(self, n, dims, seed = 4):
        # dims is an array with 6 elements
        # seed is a random seed
        self.seed = random.seed(seed)
        self.xmin = dims[0]
        self.xmax = dims[1]
        self.ymin = dims[2]
        self.ymax = dims[3]
        self.zmin = dims[4]
        self.zmax = dims[5]
        
        pPositions = []
        for i in range(n):
            pPositions.append([
                random.uniform(self.xmax - 0.5, self.xmax),
                random.uniform(self.ymin, self.ymax),
                random.uniform(self.zmin, self.zmin + 5)
            ])    
        self.particles = [particle(pPositions[i], [0, 0, 0]) for i in range(n)]
        
        self.calc_density()
        self.calc_pressure()
        self.calc_acc()
        self.calc_vel()
            
    def calc_acc(self):
        new_acc = []
        for p1 in self.particles:
            temp_acc = np.array([0,0,0], dtype = float)
            for p2 in self.particles:
                if id(p1) != id(p2):
                    temp_acc += p2.m * ((p1.pre / (p1.rho ** 2)) + (p2.pre / (p2.rho ** 2))) * (self.dgaus_kern(p1, p2))  
            p1.a=temp_acc
            
    def calc_vel(self):
        new_vels = []
        for p in self.particles:
            new_vel = p.get_dir() + (p.a * p.dt) - (p.g * p.dt)
            magnitude = np.linalg.norm(new_vel)
            if magnitude > p.vmax:
                new_vel = (p.vmax / magnitude) * new_vel
            new_vels.append(new_vel)
                
        for i in range(len(self.particles)):
            self.particles[i].d = new_vels[i]
                

    def move(self):
        for p in self.particles:
            new_pos = p.get_pos() + (p.d * p.dt)
            if new_pos[0] < self.xmin:
                new_pos[0] = self.xmin
                p.d[0] *= -0.75
                #p.d[0] *= -np.sqrt(abs(p.d[0]))
                #p.d = -0.75 * p.d
            if new_pos[0] > self.xmax:
                new_pos[0] = self.xmax
                p.d[0] *= -0.75
                #p.d[0] *= -np.sqrt(abs(p.d[0]))
                #p.d = -0.75 * p.d
            if new_pos[1] < self.ymin:
                new_pos[1] = self.ymin
                p.d[1] *= -0.75
                #p.d[1] *= -np.sqrt(abs(p.d[1]))
                #p.d = -0.75 * p.d
            if new_pos[1] > self.ymax:
                new_pos[1] = self.ymax
                p.d[1] *= -0.75
                #p.d[1] *= -np.sqrt(abs(p.d[1]))
                #p.d = -0.75 * p.d
            if new_pos[2] < self.zmin:
                new_pos[2] = self.zmin
                p.d[2] *= -0.75
                #p.d[2] *= -np.sqrt(abs(p.d[2]))
                #p.d = -0.75 * p.d
            if new_pos[2] > self.zmax:
                new_pos[2] = self.zmax
                p.d[2] *= -0.75
                #p.d[2] *= -np.sqrt(abs(p.d[2]))
                #p.d = -0.75 * p.d
            p.set_pos(new_pos)
                           
    def calc_density(self):
        new_dens = []
        for p in self.particles:
            temp = 0
            for p2 in self.particles:
                if id(p) != id(p2):
                    temp += p2.m * self.gaus_kern(p, p2)
            new_dens.append(temp)

        for i in range(len(self.particles)):
            self.particles[i].rho = new_dens[i]
               
    def calc_pressure(self):
        for p in self.particles:
            p.pre = (((0.001**2) * 1)/7) * ((p.rho / 1)**7 - 1)
            
    def gaus_kern(self, p1, p2, h = 1, d = 3):
        dir_vec = p1.get_pos() - p2.get_pos()
        dist_between = np.linalg.norm(dir_vec)
        return ((h * (np.pi ** (1/2))) ** -d) * (np.e ** (-(dist_between ** 2) / (h ** 2)))
    
    def dgaus_kern(self, p1, p2, h = 1, d = 3):
        dir_vec = p1.get_pos() - p2.get_pos()
        dist_between = np.linalg.norm(dir_vec)
        return dir_vec * (2 * (h ** -2)) * ((h * (np.pi ** (1/2))) ** -d) * (np.e ** (-(dist_between ** 2) / (h ** 2)))
                    
    # getter functions to get things
    
    def get_boundaries(self):
        return np.array([self.xmin, self.xmax, self.ymin, self.ymax, self.ymin, self.ymax])
        
    def get_pos_n(self, n):
        return self.particles[n].get_pos()
    
    def get_dir_n(self, n):
        return self.particles[n].get_dir()
    
    def get_pre_n(self, n):
        return self.particles[n].get_pre()
    
    def get_specific_accel(self, n):
        return self.particles[n].get_acc()
            
    def get_acc(self):
        for p in self.particles:
            print(p.get_acc())
            
    def get_vel(self):
        for p in self.particles:
            print(p.get_dir())
            
    def get_all_pos(self):
        all_pos = []
        for i in self.particles:
            all_pos.append(i.get_pos())
        return all_pos
        
print("Particle array class built!")

In [ ]:
# testing - delete this entire block in the final notebook!

c3 = np.array([1, 0, 0])
c4 = np.array([3, 2, 1])
print(c3 - c4)

c5 = particle(np.array([1, 0, 0]), [3, 2, 1])
c6 = particle(np.array([3, 2, 1]), [2, 1, 1])
c5.get_acc()

#c7 = pairwise_calculator(c5, c6)
#print(c7.gaus_kern())
#print(c7.dgaus_kern())

c8 = particle(np.array([1, 0, 0]), [3, 2, 1])

print(c8.g * c8.dt)

check2 = particle_array(3, [1, 2, 3, 4, 5, 6])
print(check2.get_boundaries())
#check2.print_pre()

In [ ]:
# delete all unnecessary commented out code in the final notebook!

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

# Creating data for graph plotting
plot_particles = particle_array(250, [0, 10, 0, 3, 0, 10])
# plot_particles.print_pos()
duration = 1
dt = 0.01
table = {}
def memoize():
    global table, duration, dt
    current_time = 0
    while current_time <= duration:
        table[current_time] = plot_particles.get_all_pos()
        plot_particles.move()
        plot_particles.calc_density()
        plot_particles.calc_pressure()
        plot_particles.calc_acc()
        plot_particles.calc_vel()
        #temp = []
        #for p in plot_particles.particles:
        #    temp.append(list(p.get_pos()))
        
        # table[current_time] = plot_particles.get_all_pos()
        current_time += dt
    return table
x = memoize()
print("Values generated!")

In [ ]:
%matplotlib notebook
##############################################################
#The creation of my figure                                   #
fig = plt.figure()                                           #
ax = fig.add_subplot(111, projection='3d')                   #
#Creating a global variable scat to store particle positions #
scat = ax.scatter([], [], [])                                #
##############################################################

my_timer = 0 
dt = 0.01

def init():
    """
    Creates my starting particle positions on the graph
    """
    ax.clear()
    ax.set_xlim3d([0, 10])
    ax.invert_xaxis()
    ax.set_ylim3d([0, 10])
    ax.set_zlim3d([0, 10])
    scat = ax.scatter(
        [particle[0] for particle in table[my_timer]],
        [particle[1] for particle in table[my_timer]],
        [particle[2] for particle in table[my_timer]]
    )
    return scat,


def animate(frame):
    global my_timer, dt
        
    scat._offsets3d = (
        [particle[0] for particle in table[my_timer]],
        [particle[1] for particle in table[my_timer]],
        [particle[2] for particle in table[my_timer]]
    )
    my_timer += dt
    return scat,

#Interval here delays the time per frame shown
ani = FuncAnimation(fig, animate, frames=duration, init_func=init, blit=False, interval = 250)
plt.show()